In [ ]:
import asyncio
import time
from pathlib import Path

from webgpu import proxy
from webgpu.canvas import Canvas
from webgpu.render_object import *
from webgpu.triangles import *
from webgpu.utils import init_device

from webgpu.webgpu_api import *

from IPython.display import Javascript, display
time.sleep(0.1)
js_code = Path("../webgpu/proxy.js").read_text()

display(Javascript(js_code))
display(Javascript("window._webgpu_element = element;"))

while not proxy.remote._connected_clients:
    time.sleep(0.1)

loop = asyncio.new_event_loop()
device = await init_device()

html_canvas = js.document.createElement("canvas")
html_canvas.id = 1
html_canvas.width = 512
html_canvas.height = 512
html_canvas.style = "background-color: #d0d0d0"
#div_root = js.document.getElementById("root")
div_root = js._webgpu_element
div_root.appendChild(html_canvas)
proxy.remote.on_canvas_resize(html_canvas)

canvas = Canvas(device, html_canvas)

triangle = TriangulationRenderer([0, 0, 0, 1, 0, 0, 0, 1, 0])

def render(t=0):
    triangle.options.update_buffers()
    encoder = device.createCommandEncoder()
    triangle.render(encoder)
    device.queue.submit([encoder.finish()])
    js.patchedRequestAnimationFrame(
        device.handle._id, canvas.context._id, canvas.target_texture._id
    )


options = RenderOptions(canvas, render)

triangle.options = options
triangle.options.update_buffers()
triangle.update()

pmin, pmax = triangle.get_bounding_box()

print("pmin", pmin)
print("pmax", pmax)
camera = options.camera
camera.transform._center = 0.5 * (pmin + pmax)
camera.transform._scale = 2 / np.linalg.norm(pmax - pmin)

print(camera.transform.mat)
print("center", camera.transform._center)
print("scale", camera.transform._scale)


if not (pmin[2] == 0 and pmax[2] == 0):
    camera.transform.rotate(30, -20)
camera._update_uniforms()

camera.register_callbacks(canvas.input_handler, render)

render_proxy = create_proxy(
    render,
)

# js.requestAnimationFrame(render_proxy)
render(0)
